In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import simple_soil
import flopy.plot.styles as styles

In [ ]:
solution_df = pd.DataFrame.from_dict(
    {
        "infiltration time": [
            0,
            0.25,
            0.5,
            0.75,
            1.0,
            1.25,
        ],
        "F": [
            0.0,
            0.4735,
            0.6745,
            0.8307,
            0.9638,
            1.082,
        ],
        "f": [
            0.0,
            0.963,
            0.691,
            0.5707,
            0.4988,
            0.4498,
        ],
    },
).set_index("infiltration time")
solution_df

In [ ]:
results_dict = {
    "infiltration time": [],
    "iteration": [],
    "error": [],
    "F": [],
    "f": [],
}

In [ ]:
F_t = 0.0
f_t = 0.0
infiltration_time = 0.0
delta_t = 0.25  # hrs
theta_sat = 0.479
psi = 29.22  # cm
K = 0.05  # cm/hr
initial_saturation = 0.30  # fraction
theta_eff = 0.423
delta_F = 1.0e-4

In [ ]:
theta_wp = theta_sat - theta_eff

In [ ]:
theta = initial_saturation * (theta_sat - theta_wp) + theta_wp
theta0 = initial_saturation * (theta_sat - theta_wp) + theta_wp
theta

In [ ]:
delta_theta = theta_sat - theta
delta_theta

In [ ]:
def residual(F):
    v = abs(psi) * delta_theta
    return F - v * np.log(1.0 + F / v) - K * infiltration_time

In [ ]:
def derivative(F):
    return (residual(F + delta_F) - residual(F)) / delta_F

In [ ]:
def green_ampt_infiltration(F):
    v = abs(psi) * delta_theta
    if F == 0.0:
        f = 0.0
    else:
        f = K * ((v / F) + 1)
    return f

In [ ]:
def fill_results(iteration, error, F_t, f_t):
    results_dict["infiltration time"].append(infiltration_time)
    results_dict["iteration"].append(iteration)
    results_dict["error"].append(error)
    results_dict["F"].append(F_t)
    results_dict["f"].append(f_t)

In [ ]:
def solve(F_t):
    iter, F_t, error, converged = simple_soil.utils.newton_raphson(
        residual, derivative, F_t
    )
    infiltration_rate = green_ampt_infiltration(F_t)
    fill_results(iter, error, F_t, green_ampt_infiltration(F_t))

In [ ]:
np.arange(0, 2, 0.25)

In [ ]:
for infiltration_time in np.arange(0.0, 3.0 + delta_t, delta_t):
    solve(F_t)

In [ ]:
df = pd.DataFrame.from_dict(
    results_dict,
).set_index("infiltration time")

In [ ]:
df

In [ ]:
# matplotlib 2 panel mosaic
mosaic = """
    AB
    """
line_dict = {"lw": 1.0, "color": "black"}
solution_dict = {"lw": 0, "marker": "o", "mfc": "white", "mec": "black"}

In [ ]:
with styles.USGSPlot():
    fig = plt.figure(layout="constrained", figsize=(6, 3))
    ax_dict = fig.subplot_mosaic(mosaic)

    ax = ax_dict["A"]
    df["f"].plot(ax=ax, **line_dict)
    solution_df["f"].plot(ax=ax, **solution_dict)
    ax.set_ylabel("Infitration rate (cm/hr)")

    ax = ax_dict["B"]
    df["F"].plot(ax=ax, **line_dict)
    solution_df["F"].plot(ax=ax, **solution_dict)
    ax.set_ylabel("Cumulative Infitration (cm)")

Test simple_soil Green Ampt Infiltration function

In [ ]:
ga = simple_soil.utils.GreenAmpt(
    theta_sat,
    K,
    "cm",
    soil="silty clay",
)

In [ ]:
ga_results_dict = {
    "infiltration time": [],
    "iterations": [],
    "error": [],
    "F": [],
    "f": [],
}
dt = delta_t * 1.0
for infiltration_time in np.arange(0.0, 3.0 + dt, dt):
    ga.set_infiltration_time(infiltration_time)
    ga.infiltration(K, theta, theta0)
    ga_results_dict["infiltration time"].append(infiltration_time)
    ga_results_dict["iterations"].append(ga.iterations)
    ga_results_dict["error"].append(ga.error)
    ga_results_dict["F"].append(ga.F_t)
    ga_results_dict["f"].append(ga.f_t)

In [ ]:
ga_df = pd.DataFrame.from_dict(
    ga_results_dict,
).set_index("infiltration time")

In [ ]:
ga_df

In [ ]:
with styles.USGSPlot():
    fig = plt.figure(layout="constrained", figsize=(6, 3))
    ax_dict = fig.subplot_mosaic(mosaic)

    ax = ax_dict["A"]
    ga_df["f"].plot(ax=ax, **line_dict)
    solution_df["f"].plot(ax=ax, **solution_dict)
    ax.set_ylabel("Infitration rate (cm/hr)")

    ax = ax_dict["B"]
    ga_df["F"].plot(ax=ax, **line_dict)
    solution_df["F"].plot(ax=ax, **solution_dict)
    ax.set_ylabel("Cumulative Infitration (cm)")